In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from qvix import get_qvix, update_qvix, get_hist_vol

In [ ]:
qvix = update_qvix()

In [ ]:
qvix_df = get_qvix()
# qvix_df.describe()

In [ ]:
fig = px.box(qvix_df, y='high', points='all')
fig.show()

In [ ]:
hist_vol = get_hist_vol()
columns = [name for name in hist_vol.columns if name.startswith('HV')]
quantile_df = hist_vol[columns].quantile(np.arange(.1, 1.1, .1))
quantile_df.index = list(range(10, 110, 10))

In [ ]:
quantile_df

In [ ]:
fig = go.Figure()

for index, name in quantile_df.iterrows():
    fig.add_trace(go.Scatter(x=columns, y=quantile_df.loc[index].values, mode='lines+markers', name=str(index)))

fig.show()